In [1]:
import src.lightning as l

import torch
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [14]:
L=10
N=10000
batch_size=200
checkpoint_path='./tensorboard/synth'

In [19]:
model = l.LightningSeq2Seq(L, L, warmup=400, N=2, factor=.1)

In [16]:
train = data.DataLoader(l.SynthDataset(L, N), batch_size=batch_size)
val = data.DataLoader(l.SynthDataset(L, N // 100), batch_size=batch_size)

In [20]:
trainer = pl.Trainer(
    default_root_dir=checkpoint_path,
    gpus=0,
    max_epochs=25,
    callbacks=[
        # Save the best checkpoint based on the maximum val_acc recorded. Saves only weights and not optimizer
        ModelCheckpoint(
            save_weights_only=True, mode="max", monitor="val_loss"
        ),
        # Log learning rate every epoch
        LearningRateMonitor("step")
    ],
    fast_dev_run=False
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [21]:
trainer.fit(model, train, val)


  | Name        | Type             | Params | In sizes         | Out sizes 
---------------------------------------------------------------------------------
0 | model       | TransformerModel | 14.7 M | [[1, 4], [1, 4]] | [1, 4, 10]
1 | loss_module | Seq2SeqLoss      | 0      | ?                | ?         
---------------------------------------------------------------------------------
14.7 M    Trainable params
0         Non-trainable params
14.7 M    Total params
58.921    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [22]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [23]:
%tensorboard --logdir ./tensorboard/synth

In [72]:
import torch.nn as nn

def greedy_decode(self, src, start_symbol, max_len=50, padding_idx=0, repeat_allowed=False):
    memory = self.transformer.encoder(self.src_embed(src), src_key_padding_mask=padding_mask(src, padding_idx))
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)

    for _ in range(max_len-1):
        out = self.transformer.decoder(self.trg_embed(ys), memory,
                                       tgt_mask=nn.Transformer.generate_square_subsequent_mask(ys.size(-1)),
                                       tgt_key_padding_mask=None)
        prob = self.generator(out[:, -1])
        if not repeat_allowed:
            prob[0, ys[0][-1]] = -100 # do not return the same word as the last word in the input
        
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

In [78]:
greedy_decode(model.model, torch.tensor([[1,3,5,7,9,2,4,6,8]]), 1, 9, repeat_allowed=True)

tensor([[1, 3, 5, 7, 9, 2, 4, 6, 8]])

In [66]:
x[:,-1].shape

torch.Size([1, 512])